In [1]:
"""Functions for plugging into Pipulate-frameworks for conducting SEO investigations."""

'Functions for plugging into Pipulate-frameworks for conducting SEO investigations.'

In [2]:
import requests, re
from collections import namedtuple
from html.parser import HTMLParser
from datetime import date, time, datetime, timedelta
Response = namedtuple('Response', 'ok status_code text') # Shape the standard pipulate response object

In [3]:
class date_markers():
    def __init__(self):
        shift = 3
        today = date.today()
        day = today - timedelta(days=shift)
        week = day.isocalendar()[1]
        year = day.year
        first_day_this_month = day.replace(day=1)
        first_full_week_of_year = date(year,1,1)
        if(first_full_week_of_year.weekday()>3):
            first_full_week_of_year = first_full_week_of_year+timedelta(7-first_full_week_of_year.weekday())
        else:
            first_full_week_of_year = first_full_week_of_year - timedelta(first_full_week_of_year.weekday())
        prior_weeks_of_year = timedelta(days = (week-1)*7)
        week_start = first_full_week_of_year + prior_weeks_of_year
        week_end = first_full_week_of_year + prior_weeks_of_year + timedelta(days=6)
        week_start = week_start + timedelta(days=-1)
        week_end = week_end + timedelta(days=-1)
        month_end = first_day_this_month - timedelta(days=1)
        month_start = month_end.replace(day=1)
        month_days = (month_end - month_start).days+1
        start_90_days = day - timedelta(days=90)
        end_90_days = day
        props = ['shift','first_full_week_of_year',
                 'today','day','year',
                 'first_day_this_month',
                 'prior_weeks_of_year',
                 'month_start','month_end','month_days',
                 'week','week_start','week_end',
                 'start_90_days','end_90_days']
        for prop in props:
            command = "self.%s = %s" % (prop, prop)
            exec(command)

dates = date_markers()
['%s: %s' % (x, eval('dates.%s' % x)) for x in dir(dates) if x[0] != '_']

['day: 2016-08-14',
 'end_90_days: 2016-08-14',
 'first_day_this_month: 2016-08-01',
 'first_full_week_of_year: 2016-01-04',
 'month_days: 31',
 'month_end: 2016-07-31',
 'month_start: 2016-07-01',
 'prior_weeks_of_year: 217 days, 0:00:00',
 'shift: 3',
 'start_90_days: 2016-05-16',
 'today: 2016-08-17',
 'week: 32',
 'week_end: 2016-08-13',
 'week_start: 2016-08-07',
 'year: 2016']

In [4]:
def todate(adate):
    if isinstance(adate, date):
      adate = str(adate)
    return datetime.strptime(adate, "%Y-%m-%d")

def fmdate(dobj):
   return time.strftime('%Y-%m-%d')

## Decorator to simplify functions that need the HTML of a URL-fetch as an argument

In [5]:
def url(passed_in_func):
    """Decorator for functions like Title to pre-extract html text."""
    def requests_wrapper(**row_dict):
        html = row_dict['response'].text
        return passed_in_func(html=html)
    return requests_wrapper

## Generic all-purpose helper-functions to be called from within all other functions

In [6]:
class MLStripper(HTMLParser):
    """http://stackoverflow.com/questions/753052/strip-html-from-strings-in-python"""
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [7]:
def normalize_whitespace(string):
    return re.sub('[\s\r\n]+', ' ', string)

In [8]:
def extract_text_node(html, tag):
    """Returns text node string for tags like Title. Simplifies common scraping functions."""
    pattern = r'<{0}\s?>(.*?)</{0}\s?>'.format(tag.lower())
    compiled = re.compile(pattern=pattern, flags=re.DOTALL)
    matches = compiled.findall(string=html)
    if matches:
        text = matches[0].strip()
        return Response(ok=True, status_code='200', text=text)
    else:
        return Response(ok=True, status_code='200', text=None)

## Pipulate Functions that can be used as column-labels of spreadsheets

In [9]:
@url
def Title(html):
    return extract_text_node(html=html, tag="title")

In [10]:
@url
def breadcrumb(html):
    pattern = '<ul .*?(class|id)="breadcrumb.*?>(?P<return>.*?)</ul>'.format("title")
    compiled = re.compile(pattern=pattern, flags=re.DOTALL)
    matches = compiled.search(string=html)
    if hasattr(matches, 'group'):
        text = matches.group('return')
    else:
        return Response(ok=True, status_code='200', text=None)
    scrubbed_text = normalize_whitespace(strip_tags(text))
    path_list = scrubbed_text.split("/")
    trail = '/'.join([x.strip() for x in path_list])
    return Response(ok=True, status_code='200', text=trail)

In [11]:
#if __name__ == '__main__':
    #test_url = 'http://mikelev.in/'
    #response = requests.get(test_url)
    #print(Title(url=test_url, response=response))
    #test_url = 'noyb'
    #response = requests.get(test_url)
    #print(breadcrumb(**{'response' : response}))